In [1]:
using Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("BenchmarkTools")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
using DataFrames
using CSV
using BenchmarkTools

In [3]:
Threads.nthreads()

12

## Download from Storj URL and unzip data

In [4]:
backblaze_url = "https://link.storjshare.io/s/jxw5lxqhbtfl6b2m3rshekdvfeba/external/files/csv/data_Q4_2022.zip?download=1"
review_url = "https://link.storjshare.io/s/jxk4ajvng23ltptp2ochuzmqfhna/external/files/csv/rotten_tomatoes_movie_reviews.csv.zip?download=1"
steam_url = "https://link.storjshare.io/s/jxim2lkpinczxi33rij2mtnfzn7a/external/files/csv/recommendations.csv.zip?download=1"

"https://link.storjshare.io/s/jxim2lkpinczxi33rij2mtnfzn7a/external/files/csv/recommendations.csv.zip?download=1"

In [5]:
function download_and_unzip(storj_url::String, fname::String)

    to_dir = joinpath(@__DIR__, "data")
    
    if ~isdir(to_dir)
        mkpath(to_dir)
    end
    
    file_path = joinpath(to_dir, fname)
    
    if ~isfile(file_path)
        # Fetch file
        run(`wget -q -O $file_path $storj_url`)
        
        # Unzip file        
        run(`unzip -qq $file_path -d $to_dir`)
    else
        @info "Database file already exists, skipping all actions!"
    end   
    
end

download_and_unzip (generic function with 1 method)

In [6]:
download_and_unzip(backblaze_url, "data_Q4_2022.zip");
download_and_unzip(review_url, "rotten_tomatoes_movie_reviews.csv.zip");
download_and_unzip(steam_url, "recommendations.csv.zip");

[ Info: Database file already exists, skipping all actions!
[ Info: Database file already exists, skipping all actions!
[ Info: Database file already exists, skipping all actions!


## Read CSV

In [7]:
function read_csv(fname::String)
    
    return CSV.read(fname, DataFrame)  
    
end

read_csv (generic function with 1 method)

# Backblaze data
---

## Read CSV

In [8]:
hdd_csv = "data/data_Q4_2022/2022-10-01.csv"

"data/data_Q4_2022/2022-10-01.csv"

In [9]:
b_hdd = @benchmarkable read_csv(hdd_csv) samples=100 seconds=60

Benchmark(evals=1, seconds=60.0, samples=100)

In [10]:
run(b_hdd)

BenchmarkTools.Trial: 100 samples with 1 evaluation.
 Range (min … max):  324.333 ms …    3.328 s  ┊ GC (min … max): 0.00% … 1.24%
 Time  (median):     342.182 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   374.518 ms ± 298.690 ms  ┊ GC (mean ± σ):  1.17% ± 2.09%

   ▁ ▁▁▁▁█▁▆ ▆   ▁  ▃      ▃▁   ▆                  ▁             
  ▇█▇███████▁█▇▄▇█▇▄█▇▇▇▇▁▁██▇▄▇█▄▄▁▇▄▁▄▄▁▇▇▁▁▄▄▁▄▁█▁▁▁▁▁▇▁▄▁▁▄ ▄
  324 ms           Histogram: frequency by time          384 ms <

 Memory estimate: 411.75 MiB, allocs estimate: 294914.

## Filtering

In [11]:
function get_failed_drives(df_hdd::DataFrame)
    
    return filter(row -> row.failure > 0, df_hdd)
    
end

get_failed_drives (generic function with 1 method)

In [12]:
df_hdd = read_csv(hdd_csv);

In [13]:
b_filter_hdd = @benchmarkable get_failed_drives(df_hdd) samples=300 seconds=100

Benchmark(evals=1, seconds=100.0, samples=300)

In [14]:
run(b_filter_hdd)

BenchmarkTools.Trial: 300 samples with 1 evaluation.
 Range (min … max):  13.547 ms … 449.311 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     15.759 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.790 ms ±  25.434 ms  ┊ GC (mean ± σ):  3.80% ± 9.73%

  ▂ ▁█▆                                                         
  █▇████▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▄▁▁▄ ▆
  13.5 ms       Histogram: log(frequency) by time      49.6 ms <

 Memory estimate: 7.10 MiB, allocs estimate: 461480.

## Merging

In [15]:
df_hdd_1 = read_csv("data/data_Q4_2022/2022-10-02.csv");
df_hdd_2 = read_csv("data/data_Q4_2022/2022-10-03.csv");

In [16]:
function join_df(df_hdd_1::DataFrame, df_hdd_2::DataFrame)
    
    return innerjoin(df_hdd_1, df_hdd_2, on = :serial_number, makeunique = true)
    
end

join_df (generic function with 1 method)

In [17]:
b_join_hdd = @benchmarkable join_df(df_hdd_1, df_hdd_2) samples=200 seconds=100

Benchmark(evals=1, seconds=100.0, samples=200)

In [18]:
run(b_join_hdd)

BenchmarkTools.Trial: 200 samples with 1 evaluation.
 Range (min … max):  141.625 ms … 411.514 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     146.544 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   153.597 ms ±  22.425 ms  ┊ GC (mean ± σ):  2.38% ± 5.61%

   ▄█▄                                                           
  ▄███▅▆▃▅▆▃▃▃▃▃▃▃▃▂▅▃▂▃▃▃▁▁▃▂▁▁▁▁▂▁▂▁▃▁▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▂▁▁▁▁▁▁▂ ▃
  142 ms           Histogram: frequency by time          215 ms <

 Memory estimate: 658.80 MiB, allocs estimate: 7089.

## Aggregation

In [19]:
function get_hdd_stats(df_hdd::DataFrame)
    
    return combine( groupby(df_hdd, :model, sort = true),
                    :smart_5_raw => maximum,
                    :smart_7_raw => maximum,
                    :smart_197_raw => maximum,
                    :smart_198_raw => maximum
                  )
    
end

get_hdd_stats (generic function with 1 method)

In [20]:
b_agg_hdd = @benchmarkable get_hdd_stats(df_hdd) samples=200 seconds=60

Benchmark(evals=1, seconds=60.0, samples=200)

In [21]:
run(b_agg_hdd)

BenchmarkTools.Trial: 200 samples with 1 evaluation.
 Range (min … max):   3.996 ms … 668.773 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      6.489 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.095 ms ±  46.822 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

                  ▆██▅▃                                         
  ▄▁▁▁▁▁▁▁▁▁▄▁▁▁▁▄██████▆▆█▇▄▁▁▆▁▄▁▄▁▁▆▁▄▆▆▁▄▁▁▁▄▁▁▁▁▁▁▁▄▁▄▁▁▄ ▆
  4 ms          Histogram: log(frequency) by time      12.1 ms <

 Memory estimate: 1.82 MiB, allocs estimate: 888.

# Steam data
---

## Read CSV

In [22]:
steam_csv = "data/recommendations.csv"

"data/recommendations.csv"

In [23]:
b_steam = @benchmarkable read_csv(steam_csv) samples=100 seconds=150

Benchmark(evals=1, seconds=150.0, samples=100)

In [24]:
run(b_steam)

BenchmarkTools.Trial: 100 samples with 1 evaluation.
 Range (min … max):  743.277 ms …   1.257 s  ┊ GC (min … max): 0.00% … 0.94%
 Time  (median):     778.301 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   801.040 ms ± 80.566 ms  ┊ GC (mean ± σ):  0.26% ± 0.88%

  ▂█▄                                                           
  ███▆▅▆█▅▆▅▅▃▃▄▁▃▃▁▁▁▁▃▁▃▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▃
  743 ms          Histogram: frequency by time          1.25 s <

 Memory estimate: 610.71 MiB, allocs estimate: 3447.

## Filtering

In [25]:
function get_good_games(df_steam::DataFrame)
    
    return filter(row -> row.hours > 72 && row.helpful > 100, df_steam)  
    
end

get_good_games (generic function with 1 method)

In [26]:
df_steam = read_csv(steam_csv);
size(df_steam)

(10072270, 8)

In [27]:
b_filter_steam = @benchmarkable get_good_games(df_steam) samples=100 seconds=100

Benchmark(evals=1, seconds=100.0, samples=100)

In [28]:
run(b_filter_steam)

BenchmarkTools.Trial: 76 samples with 1 evaluation.
 Range (min … max):  1.251 s …   1.437 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.322 s              ┊ GC (median):    4.75%
 Time  (mean ± σ):   1.322 s ± 21.977 ms  ┊ GC (mean ± σ):  4.47% ± 1.20%

                                   █▂ ▂ ▄█ ▂▂ ▆              
  ▄▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▄▆▄▆▁████▆██▆██▄█▆▄█▆▄▆▁█▁▄▁▄ ▁
  1.25 s         Histogram: frequency by time        1.35 s <

 Memory estimate: 608.78 MiB, allocs estimate: 39699489.

In [29]:
df_good = get_good_games(df_steam);

## Aggregation

In [30]:
function get_hours_stats_1(df_steam::DataFrame)
    
    return combine( groupby(df_steam, :app_id, sort = true),
                    :hours => minimum,
                    :hours => maximum,
                    :hours => mean,
                    :hours => sum,
                    :is_recommended => sum,
                    :helpful => sum,
                    :funny => sum
                   )
    
end

get_hours_stats_1 (generic function with 1 method)

In [31]:
b_agg_1 = @benchmarkable get_hours_stats_1(df_steam) samples=1000 seconds=60

Benchmark(evals=1, seconds=60.0, samples=1000)

In [32]:
run(b_agg_1)

BenchmarkTools.Trial: 201 samples with 1 evaluation.
 Range (min … max):  253.620 ms …   1.169 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     292.075 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   299.438 ms ± 64.622 ms  ┊ GC (mean ± σ):  1.08% ± 2.54%

                  ▇▄██▃▂ ▇▄ ▂▃▇    ▃▂                           
  ▅▆▁▅▃▃▃▅▃▁▅█▃▃▇▇█████████▇████▆▅▃██▅▆█▆▅▃▆▅▃▃▅▇▅▅▃▁▃▆▃▅▁▁▁▁▃ ▅
  254 ms          Histogram: frequency by time          349 ms <

 Memory estimate: 95.91 MiB, allocs estimate: 991.

# Rotten Tomatoes data
---

## Read CSV

In [33]:
movies_csv = "data/rotten_tomatoes_movie_reviews.csv"

"data/rotten_tomatoes_movie_reviews.csv"

In [34]:
b_movies = @benchmarkable read_csv(movies_csv) samples=100 seconds=150

Benchmark(evals=1, seconds=150.0, samples=100)

In [35]:
run(b_movies)

BenchmarkTools.Trial: 100 samples with 1 evaluation.
 Range (min … max):  486.592 ms …    1.459 s  ┊ GC (min … max):  0.00% … 63.25%
 Time  (median):     506.616 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   588.689 ms ± 211.592 ms  ┊ GC (mean ± σ):  11.36% ± 16.40%

  ██▃▂▁                                                          
  ██████▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▅▅▅▁▅▁▁▁▁▁▅▁▆▁▅▁▁▁▁▁▅▁▁▁▁▁▁▁▅▅ ▅
  487 ms        Histogram: log(frequency) by time        1.34 s <

 Memory estimate: 872.31 MiB, allocs estimate: 9491026.

## Filtering

In [36]:
function get_good_movies(df_movies::DataFrame)
    
    return filter(row -> row.isTopCritic == true && row.scoreSentiment == "POSITIVE",
                  df_movies)  
    
end

get_good_movies (generic function with 1 method)

In [37]:
df_movies = read_csv(movies_csv);
size(df_movies)

(1444963, 11)

In [38]:
b_filter_movies = @benchmarkable get_good_movies(df_movies) samples=100 seconds=100

Benchmark(evals=1, seconds=100.0, samples=100)

In [39]:
run(b_filter_movies)

BenchmarkTools.Trial: 100 samples with 1 evaluation.
 Range (min … max):  146.960 ms … 404.223 ms  ┊ GC (min … max): 0.00% …  0.00%
 Time  (median):     164.694 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   176.321 ms ±  38.192 ms  ┊ GC (mean ± σ):  6.18% ± 12.11%

       ▇█                                                        
  ▃▃▂▁▅██▇▃▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▃▂▁▁▁▂▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▂
  147 ms           Histogram: frequency by time          317 ms <

 Memory estimate: 92.25 MiB, allocs estimate: 4199792.

## Aggregation

In [40]:
function get_movies_stats(df_movies::DataFrame)
    
    cols = [:id, :reviewState, :scoreSentiment]    
    return combine(groupby(df_movies, cols , sort = true), nrow)
    
end

get_movies_stats (generic function with 1 method)

In [41]:
b_agg_movies = @benchmarkable get_movies_stats(df_movies) samples=1000 seconds=60

Benchmark(evals=1, seconds=60.0, samples=1000)

In [42]:
run(b_agg_movies)

BenchmarkTools.Trial: 377 samples with 1 evaluation.
 Range (min … max):  146.557 ms …   1.341 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     153.158 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   158.506 ms ± 61.431 ms  ┊ GC (mean ± σ):  0.51% ± 1.84%

       ▄▅ █▄ ▂▁    ▁▁                                           
  ▃▃▁▅█████████▆▅█▅██▇▆▅▅▅▄▅▃▄▄▃▅▄▁▃▁▄▂▄▂▂▃▃▂▃▂▂▁▂▁▂▁▂▁▂▂▁▁▁▂▃ ▃
  147 ms          Histogram: frequency by time          178 ms <

 Memory estimate: 49.84 MiB, allocs estimate: 10742.